## None과 독스트링을 사용해 동적인 디폴트 인자를 지정하라

종종 키워드 인자의 값으로 정적으로 정해지지 않는 타입의 값을 써야 할 때가 있다.

예를 들어 로그 메시지와 시간을 함께 출력하고 싶다고 하자. 기본적으로 함수 호출 시간을 포함하길 원한다. 함수가 호출될 때마다 디폴트 인자가 재계산된다고 가정하면 다음과 같은 접근 방법이 있다.

In [1]:
from time import sleep
from datetime import datetime

def log(message, when=datetime.now()):
    print(f'{when}: {message}')
    
log('안녕')
sleep(0.1)
log('다시 안녕!')

2023-04-28 10:13:50.336861: 안녕
2023-04-28 10:13:50.336861: 다시 안녕!


하지만 디폴트 인자는 이런 식으로 작동하지 않는다. 함수가 정의되는 시점에 datetime.now 가 단 한번만 호출되기 때문에 타임스탬프가 항상 같다.

디폴트 인자의 값은 모듈이 로드될 때 단 한 번만 평가되는데, 보통 프로그램이 시작할 때 모듈을 로드하는 경우가 많다.

이런 경우 원하는 동작을 달성하는 파이썬의 일반적인 관례는 디폴트 값으로 None을 지정하고 실제 동작을 독스트링에 문서화하는 것이다.

In [2]:
def log(message, when=None):
    """메시지와 타임스탬프를 로그에 남긴다.
    
    Args:
        meseage: 출력할 메시지,
        when: 메시지가 발생한 시각(datetime),
            디폴트 값은 현재 시간이다.
    
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')
    
log('안녕')
sleep(0.1)
log('다시 안녕!')

2023-04-28 10:16:55.574068: 안녕
2023-04-28 10:16:55.675362: 다시 안녕!


디폴트 인자 값으로 None을 사용하는 것은 인자가 가변적인 경우 특히 중요하다. 예를 들어 JSON 데이터로 인코딩된 값을 읽으려고 하는데, 데이터 디코딩에 실패하면 디폴트로 빈 딕셔너리를 반환하고 싶다.

In [5]:
import json

def decode(data, default={}):
    try:
        return json.loads(data)
    except ValueError:
        return default

이 코드의 문제점은 앞에서 본 datetime.now의 경우와 같다. 디폴트 값이 단 한 번만 평가되기 때문에 dafault에 지정된 딕셔너리가 decode 호출에 모두 공유된다.

In [6]:
foo = decode('잘못된 데이터')
foo['stuff'] = 5
bar = decode('또 잘못된 데이터')
bar['meep'] = 1

print(foo)
print(bar)

{'stuff': 5, 'meep': 1}
{'stuff': 5, 'meep': 1}


키와 값이 하나뿐인 서로 다른 딕셔너리가 두 개 출력될 것으로 예상한 독자도 있을 것이다. 하지만 한쪽 딕셔너리를 변경하면 다른 쪽 딕셔너리도 변경되는 것처럼 보인다.

foo와 bar가 모두 default 파라미터와 같기 때문이다. 둘은 모두 동일한 딕셔너리 객체다.

In [7]:
assert foo is bar

이 문제의 해법은 이 함수에 있는 키워드 인자의 디폴트 값으로 None을 지정하고 함수의 독스트링에 동작 방식을 기술하는 것이다.

In [12]:
import json

def decode(data, default=None):
    """문자열로부터 JSON 데이터를 읽어온다.
    
    Args:
        data: 디코딩할 JSON 데이터.
        default: 디코딩 실패 시 반환할 값이다.
            디폴트 값은 빈 딕셔너리이다.
    
    """
    
    try:
        return json.loads(data)
    except ValueError:
        if default is None:
            default = {}
        return default

In [14]:
foo = decode('잘못된 데이터')
print(foo)
foo['stuff'] = 5
bar = decode('또 잘못된 데이터')
bar['meep'] = 1

print(foo)
print(bar)

{}
{'stuff': 5}
{'meep': 1}


이 접근 방법은 타입 애너테이션을 사용해도 잘 작동한다.

In [15]:
from typing import Optional

def log_typed(message: str, 
             when: Optional[datetime]=None) -> None:
    """메시지와 타임스탬프를 로그에 남긴다.
    
    Args:
        meseage: 출력할 메시지,
        when: 메시지가 발생한 시각(datetime),
            디폴트 값은 현재 시간이다.
    
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

In [16]:
log_typed('안녕')
sleep(0.1)
log_typed('다시 안녕!')

2023-04-28 10:31:41.610240: 안녕
2023-04-28 10:31:41.711456: 다시 안녕!


In [17]:
Optional?

Signature:   Optional(*args, **kwds)
Type:        _SpecialForm
String form: typing.Optional
File:        ~/opt/anaconda3/lib/python3.8/typing.py
Docstring:  
Internal indicator of special typing constructs.
See _doc instance attribute for specific docs.
